<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks&para;">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective&para;">Objective</h3>
<br><strong style="color: #000000;"><em>Find and Delete unused AWS Cloudwatch Log Streams</em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Delete-Unused-AWS-Secrets"><u>Delete Unused AWS Log Streams</u></h2>
</center>
<h1 id="Steps-Overview&para;">Steps Overview</h1>
<p>1)<a href="#1" target="_self" rel="noopener"> Find unused log streams</a><br>2)<a href="#2" target="_self" rel="noopener"> Delete unused log streams</a></p>

In [ ]:
if region == None:
    region = ''
if log_stream_name and log_group_name and not region:
    raise SystemExit("Provide a region for the Logs!")
elif not log_group_name and region and log_stream_name:
    raise SystemExit("Provide a Log Group Name!")
elif not log_stream_name and region and log_group_name:
    raise SystemExit("Provide a Log Stream Name!")
elif not log_stream_name and not region and log_group_name:
    raise SystemExit("Provide a Log Stream Name and region!")
elif not log_group_name and not region and log_stream_name:
    raise SystemExit("Provide a Log Group Name and region !")

<h3 id="Filter-unused-Secrets"><a id="1" target="_self" rel="nofollow"></a>Filter unused log streams</h3>
<p>Using unSkript's Filter AWS Filter Unused Log Streams action, we will find unused log streams given a threshold number of days from their last use date.&nbsp;</p>
<blockquote>
<p>This action takes the following parameters: <code>region, threhold_days</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>unused_log_streams</code></p>
</blockquote>

In [ ]:
##
##  Copyright (c) 2023 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Optional, List, Tuple
from datetime import datetime, timedelta
import botocore.config
from unskript.connectors.aws import aws_get_paginator
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
import pprint


from beartype import beartype
@beartype
def aws_filter_unused_log_streams_printer(output):
    if output is None:
        return

    pprint.pprint(output)


@beartype
def aws_filter_unused_log_streams(handle, region: str = "", time_period_in_days: int = 30) -> Tuple:
    """aws_filter_unused_log_streams Returns an array of unused log strams for all log groups.

        :type region: string
        :param region: Used to filter the volume for specific region.

        :type time_period_in_days: int
        :param time_period_in_days: (in days) The threshold to filter the unused log strams.

        :rtype: Array of unused log strams for all log groups.
    """
    result = []
    now = datetime.utcnow()
    all_regions = [region]
    if not region:
        all_regions = aws_list_all_regions(handle)

    for reg in all_regions:
        try:
            start_time = now - timedelta(days=time_period_in_days)
            config = botocore.config.Config(retries={'max_attempts': 10})
            ec2Client = handle.client('logs', region_name=reg, config=config)
            response = aws_get_paginator(ec2Client, "describe_log_groups", "logGroups")
            for log_group in response:
                log_group_name = log_group['logGroupName']
                response1 = aws_get_paginator(ec2Client, "describe_log_streams", "logStreams",
                                            logGroupName=log_group_name,
                                            orderBy='LastEventTime',
                                            descending=True)

                for log_stream in response1:
                    unused_log_streams = {}
                    last_event_time = log_stream.get('lastEventTimestamp')
                    if last_event_time is None:
                        # The log stream has never logged an event
                        unused_log_streams["log_group_name"] = log_group_name
                        unused_log_streams["log_stream_name"] = log_stream['logStreamName']
                        unused_log_streams["region"] = reg
                        result.append(unused_log_streams)
                    elif datetime.fromtimestamp(last_event_time/1000.0) < start_time:
                        # The log stream has not logged an event in the past given days
                        unused_log_streams["log_group_name"] = log_group_name
                        unused_log_streams["log_stream_name"] = log_stream['logStreamName']
                        unused_log_streams["region"] = reg
                        result.append(unused_log_streams)
        except Exception as e:
            pass

    if len(result) != 0:
        return (False, result)
    else:
        return (True, None)


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "time_period_in_days": "int(threshold_days)",
    "region": "region"
    }''')

task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not log_stream_name and not log_group_name",
    "condition_result": true
    }''')

task.configure(outputName="unused_log_streams")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_filter_unused_log_streams, lego_printer=aws_filter_unused_log_streams_printer, hdl=hdl, args=args)

<h3 id="Create-List-of-Unused-Log-Streams&para;">Create List of Unused Log Streams</h3>
<p>This action filters regions that have no unused log streams and creates a list of those that have them.</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>all_unused_log_streams</code></p>
</blockquote>

In [ ]:
all_unused_log_streams = []
try:
    if unused_log_streams[0] == False:
        if len(unused_log_streams[1])!=0:
            all_unused_log_streams=unused_log_streams[1]
except Exception:
    for log_s in log_stream_name:
        data_dict = {}
        data_dict["region"] = region
        data_dict["log_group_name"] = log_group_name
        data_dict["log_stream_name"] = log_s
        all_unused_log_streams.append(data_dict)
print(all_unused_log_streams)

<h3 id="Delete-unused-Secrets"><a id="2" target="_self" rel="nofollow"></a>Delete unused log streams</h3>
<p>This action deleted unused log streams found in Step 1.&nbsp;</p>
<blockquote>
<p>This action takes the following parameters:&nbsp;<code>all_unused_log_streams</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Optional, Dict
import pprint


from beartype import beartype
@beartype
def aws_delete_log_stream_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_delete_log_stream(handle, log_group_name: str, log_stream_name: str, region: str) -> Dict:
    """aws_delete_log_stream Deletes a log stream.

        :type log_group_name: string
        :param log_group_name: Name of the log group.

        :type log_stream_name: string
        :param log_stream_name: Name of the log stream.

        :type region: string
        :param region: AWS Region.

        :rtype: Dict with the deleted log stream info.
    """
    try:
        log_Client = handle.client('logs', region_name=region)
        response = log_Client.delete_log_stream(
            logGroupName=log_group_name,
            logStreamName=log_stream_name)
        return response
    except Exception as e:
        raise Exception(e)


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "log_group_name": "iter.get(\\"log_group_name\\")",
    "log_stream_name": "iter.get(\\"log_stream_name\\")",
    "region": "iter.get(\\"region\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_unused_log_streams",
    "iter_parameter": ["log_group_name","log_stream_name","region"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(all_unused_log_streams)!=0",
    "condition_result": true
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_delete_log_stream, lego_printer=aws_delete_log_stream_printer, hdl=hdl, args=args)

<h3 id="Conclusion">Conclusion</h3>
<p>In this Runbook, we were able to filter unused log streams before a given threshold number of days and delete them. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>